In [1]:
import os
import sys
import pycurl
import hashlib
import sqlite3
import requests
import numpy as np
import pandas as pd
from io import BytesIO
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [10]:
def table_exists(db='document.db', table='documents'):
    con=sqlite3.connect(db)
    cur = con.cursor()
    query = f"""SELECT name FROM sqlite_master WHERE type='table' AND name="{table}";"""
    cur.execute(query)
    if len(cur.fetchall()) > 0:
        con.close()
        return True
    else: 
        con.close()
        return False

def create_table(db="document.db", table="documents"):
    # hash, filename, filepath, filetype, size, accesstime, modtime, createtime
    con=sqlite3.connect(db)
    cur = con.cursor()
    query = f"""
            CREATE TABLE '{table}' (
                hash TEXT NOT NULL, 
                filename TEXT NOT NULL, 
                filepath TEXT NOT NULL,
                filetype TEXT,
                size REAL,
                accesstime TEXT,
                modtime TEXT,
                createtime TEXT,
                metadata TEXT,
                summary TEXT,
                topics TEXT
            );
            """
    try:
        cur.execute(query)
        con.commit()
        con.close()
        return None
    except Exception as e: 
        con.close()
        return e
    
def drop_table(db="document.db", table="documents"):
    con=sqlite3.connect(db)
    cur = con.cursor()
    query = f"DROP TABLE '{table}';"
    try:
        cur.execute(query)
        con.commit()
        con.close()
        return None
    except Exception as e: 
        con.close()
        return e
    
def gen_md5(file):
    return hashlib.md5(open(file,'rb').read()).hexdigest()

def _metadata(tika_url = "http://localhost:9998/meta", file=None):
    if not file:
        return 0
    headers = {"Content-type": "application/pdf", 
           "X-Tika-PDFocrStrategy": "no_ocr",
           "X-Tika-Timeout-Millis": "300000",
           "file": file}
    r = requests.put(tika_url, data=open(file, mode='rb'), headers=headers)
    return r.text

def _metadata_curl(tika_url = "http://localhost:9998/meta", file=None):
    if not file:
        return 0
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, tika_url)
    c.setopt(c.UPLOAD, 1)
    c.setopt(c.HTTPHEADER, ['Content-type: application/pdf','X-Tika-PDFocrStrategy: no_ocr'])
    f = open(file, 'rb')
    c.setopt(c.READDATA, f)
    c.setopt(c.VERBOSE, False)
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()
    f.close()
    return buffer.getvalue().decode("utf-8")

def get_metadata(tika_url = "http://localhost:9998/meta", file=None):
    try:
        metadata = _metadata(file=file)
        return metadata
    except Exception as e1:
        try:
            metadata = _metadata_curl(file=file)
            return metadata
        except Exception as e2:
            print(f"bailing out: {file}")
            return -1

def indexer(base_path = "."):
    fn, fp, ft, sz, at, mt, ct, md5, metadata, summary, topics = [], [], [], [], [], [], [], [], [], [], []
    for root, dirs, files in os.walk(base_path, topdown=False):
        for name in files:
            f = os.path.join(root, name)
            stat = os.stat(f) #st_size, st_atime, st_mtime, st_ctime
            fn.append(name)
            fp.append(f)
            md5.append(gen_md5(f))
            ft.append(os.path.splitext(f)[-1].split('.')[-1])
            sz.append(stat.st_size)
            at.append(datetime.fromtimestamp(stat.st_atime))
            mt.append(datetime.fromtimestamp(stat.st_mtime))
            ct.append(datetime.fromtimestamp(stat.st_ctime))
            metadata.append(None)
            summary.append(None)
            topics.append(None)
    return {"hash":md5, "filename":fn, "filepath":fp, "filetype":ft, "size":sz, "accesstime":at, 
            "modtime":mt, "createtime":ct, "metadata":metadata, "summary":summary, "topics":topics}


def dump_table(db="document.db", table="documents", values = None):
    con=sqlite3.connect(db)
    cur = con.cursor()
    query = f"SELECT * FROM '{table}';"
    cur.execute(query)
    res = cur.fetchall()
    con.close()
    return res

def insert(db="document.db", table="documents", values = None):
    if not values:
        return 0
    con=sqlite3.connect(db)
    cur = con.cursor()
    query = f""" INSERT INTO '{table}' 
                 (hash, filename, filepath, filetype, size, accesstime, modtime, createtime, metadata, summary, topics)
             VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    try:
        res = cur.executemany(query, values)
        con.commit()
        return 0
    except Exception as e:
        return e
    return res

def run_indexer():
    d = indexer(base_path = ".")
    file_info = list(zip(d["hash"], d["filename"], d["filepath"], d["filetype"], 
                     d["size"], d["accesstime"], d["modtime"], 
                     d['createtime'], d['metadata'], d['summary'], d['topics']))
    return file_info


In [17]:
#drop_table()

In [18]:
create_table()
insert(values=run_indexer())

0

In [19]:
db="document.db"
table="documents"

con=sqlite3.connect(db)
cur = con.cursor()
query = f"""SELECT filename, filepath, hash FROM '{table}' WHERE metadata IS NULL"""
cur.execute(query)
res = cur.fetchall()
con.close()


In [22]:
res[:5]

[('Hands-On Machine Learning with Scikit-Learn _ Tensorflow [Geron].pdf',
  './Books/Hands-On Machine Learning with Scikit-Learn _ Tensorflow [Geron].pdf',
  '5fb4755551e96acf31bd64d9585f7760'),
 ('Fundamentals of Differential Equations [Nagle, Saff, Snider] 8th Edition.pdf',
  './Books/Fundamentals of Differential Equations [Nagle, Saff, Snider] 8th Edition.pdf',
  '3b0ff6478a51cf2e0689c32d98c04660'),
 ('Mechanics of Materials (Solids) [Hibbeler] 10th Edition.pdf',
  './Books/Mechanics of Materials (Solids) [Hibbeler] 10th Edition.pdf',
  '4acd24e97a0540d9e09a7997dd290621'),
 ('Engineering Mechanics_ Dynamics [Fowler, Bedford].pdf',
  './Books/Engineering Mechanics_ Dynamics [Fowler, Bedford].pdf',
  'cb725acfb5e715d280c3688e4a1131a2'),
 ('Calculus_ Early Transcendentals [James Stewart] 7th Edition.docx',
  './Books/Calculus_ Early Transcendentals [James Stewart] 7th Edition.docx',
  '31ffb1b13dc48e9924981845e891251d')]

In [ ]:
# headers = {
#     "X-Tika-OCRLanguage": "eng",
#     "X-Tika-OCRTimeout": "300",
#     "X-Tika-PDFocrStrategy": "auto"
# }
# headers = {"Content-type": "application/pdf", 
#            "X-Tika-OCRLanguage": "eng", 
#            "X-Tika-PDFocrStrategy": "auto", 
#            "X-Tika-OCRTimeout": "500", 
#            "file": file}

In [ ]:
# https://stackoverflow.com/questions/3634984/insert-if-not-exists-else-update
# https://stackoverflow.com/questions/418898/sqlite-upsert-not-insert-or-replace
# https://stackoverflow.com/questions/15277373/sqlite-upsert-update-or-insert

In [ ]:
# df = pd.DataFrame(d)
# df[['filename', 'hash']].head(5)

In [ ]:
#gen_md5(file=file_info[1][-2])

In [ ]:
# https://docs.python.org/3/library/sqlite3.html
# con = sqlite3.connect("document.db")
# cur = con.cursor()

In [ ]:
# for root, dirs, files in os.walk(".", topdown=False):
#     for name in files:
#         print(os.path.join(root, name))
#     for name in dirs:
#         print(os.path.join(root, name))

# file_size = os.stat('d:/file.jpg')
# print("Size of file :", file_size.st_size, "bytes")

# Cross-platform way to get file modification time in Python. 
# It returns the Unix timestamp of when the file was last modified.
# os.path.getmtime(path)

# path = 'D:\Work TP.py'
# # declaring the variable to get the result
# result = os.path.splitext(path)

# print('Path:', result[0])
# print('Extension:', result[1])

In [ ]:
# A = [1,2,3,4,5,6,7,8,9]
# B = [9,8,7,6,5,4,3,2,1]
# pd.DataFrame({"A": A, "B": B})

In [ ]:
# SCHEMA
# file name, file path, file type, size, atime, mtime, ctime

In [ ]:
# con = sqlite3.connect("document.db")
# cur = con.cursor()
# table = "documents"
# query = f"""SELECT name FROM sqlite_master WHERE type="table" AND name="{table}";"""
# cur.execute(query)
# cur.fetchall()

In [ ]:
# https://www.sqlitetutorial.net/sqlite-create-table/
# https://www.sqlitetutorial.net/sqlite-index/
# https://www.sqlitetutorial.net/sqlite-alter-table/

# CREATE UNIQUE INDEX idx_contacts_email ON contacts (email)

In [ ]:
# # SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';
# query = "SELECT name FROM sqlite_master WHERE type='table' AND name='documents';"
# cur.execute(query)
# cur.fetchall()

In [ ]:
# df = pd.DataFrame(d)
# df.to_sql(name='documents', con=con, index=False, if_exists='replace')

In [ ]:
# pd.read_sql('select * from documents', con)
#pd.read_sql('select filename, filepath from documents', con)

In [ ]:
# # cur.executemany('insert into Counts (email, count) values (?,?)',Counts)
# # conn.commit()

# db = 'document.db'
# table='documents'

# con=sqlite3.connect(db)
# cur = con.cursor()

# query = f""" INSERT INTO '{table}' 
#                  (hash, filename, filepath, filetype, size, accesstime, modtime, createtime, summary)
#              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"""

# res = cur.executemany(query, file_info)
# con.commit()

In [ ]:
# # This works

# # curl -X PUT --data-binary @'An Introduction to Biomechanics - Jay D Humphrey- Sherry L O’Rourke.pdf' http://localhost:9998/meta --header "Content-type: application/pdf" --header "X-Tika-PDFocrStrategy: no_ocr"

# file = "./Springer/An Introduction to Biomechanics - Jay D Humphrey- Sherry L O’Rourke.pdf"

# buffer = BytesIO()

# c = pycurl.Curl()
# c.setopt(c.URL, 'http://localhost:9998/meta')
# c.setopt(c.UPLOAD, 1)
# c.setopt(c.HTTPHEADER, ['Content-type: application/pdf','X-Tika-PDFocrStrategy: no_ocr'])
# f = open(file, 'rb')
# c.setopt(c.READDATA, f)
# c.setopt(c.VERBOSE, False)

# c.setopt(c.WRITEDATA, buffer)

# c.perform()
# c.close()
# # File must be kept open while Curl object is using it
# f.close()

# body = buffer.getvalue()

# print(f"RESULT =====>: {body.decode('iso-8859-1')}")
